# **Install Required Dependencies**

In [1]:
!pip install fastapi uvicorn pyngrok whisper torch
!pip install git+https://github.com/openai/whisper.git
!pip install torch transformers fastapi uvicorn pyngrok
!pip install python-multipart
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━

# **Authenticate Ngrok in Colab**

In [2]:
!ngrok config add-authtoken 2szDRXwf1lzswjmg821h9PeXjlG_6zbf7US4YVbZ4mcspLmay

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


# **Import Libraries**

In [3]:
import re
import os
import torch
import uvicorn
import threading
import uuid
import subprocess
import whisper
from whisper import load_model
from pydantic import BaseModel
from pyngrok import ngrok
from pathlib import Path
from fastapi import FastAPI, UploadFile, File, HTTPException, Query
from transformers import MarianMTModel, MarianTokenizer
from torch.quantization import quantize_dynamic
from fastapi.responses import JSONResponse

# **Load Model whisper && translation model**

In [4]:
# Check if CUDA (GPU) is available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the Whisper model
model = whisper.load_model("small", device=device)

# Load and quantize translation model (English to Arabic)
model_name = "Helsinki-NLP/opus-mt-tc-big-en-ar"
modelNmt = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)
modelNmt = quantize_dynamic(modelNmt, {torch.nn.Linear}, dtype=torch.qint8)

100%|███████████████████████████████████████| 461M/461M [00:18<00:00, 26.8MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/478M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/337 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/806k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/916k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.21M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

# **Create the FastAPI App**

In [5]:
# Initialize FastAPI app
app = FastAPI()

# **transcribe video API**

## **extract audio**

In [6]:
def extract_audio(video_path: str) -> str:
    """Extracts audio from a video file using FFmpeg and saves it as an MP3."""
    audio_file = str(Path(video_path).with_suffix(".mp3"))

    subprocess.run(
        ["ffmpeg", "-i", video_path, "-q:a", "0", "-map", "a", audio_file],
        check=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )

    return audio_file

##**transcribe audio to srt format**

In [7]:
def transcribe_audio_to_srt_format(audio_path: str) -> str:
    """Transcribes an audio file using Whisper and returns a subtitle-formatted text."""
    result = model.transcribe(audio_path)

    transcription = ""
    total_segments = len(result["segments"])
    for index, segment in enumerate(result["segments"]):
        start = segment["start"]
        end = segment["end"]
        text = segment["text"]
        transcription += f"[{start:.2f} - {end:.2f}] {text}\n"

    return transcription

## **Translate Transcribe**

In [8]:
def translate_to_arabic(text: str) -> str:
    """Translate English transcription to Arabic using MarianMT."""
    try:
        # Correct timestamp format using regex (fix colons and missing decimals)
        text = re.sub(r'(\d+):(\d+)', r'\1.\2', text)
        text = re.sub(r'(\d+)\.(\d{2})\.(\d{2})', r'\1.\2\3', text)

        # Regex pattern to capture timestamps and text separately
        pattern = r'\[(\d{1,3}\.\d{2})\s*-\s*(\d{1,3}\.\d{2})\]\s*(.*)'

        translated_lines = []
        for match in re.finditer(pattern, text):
            start_time, end_time, content = match.groups()

            if content.strip():
                # Tokenize and translate
                encoded_text = tokenizer([content], return_tensors="pt", padding=True, truncation=True)
                output_ids = modelNmt.generate(encoded_text['input_ids'], num_beams=1, length_penalty=1.0)
                translated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

                # Store translated line with timestamps
                translated_lines.append(f"[{start_time} - {end_time}] {translated_text}")

        return "\n".join(translated_lines)

    except Exception as e:
        return f"Translation error: {str(e)}"


## **EndPiont For transcribe**

In [9]:
@app.post("/transcribe/")
async def transcribe_video(file: UploadFile = File(...),
          translate: bool = Query(True, description="Set to True to translate to Arabic")):
    """API endpoint to handle video file uploads and return SRT-formatted text, with optional Arabic translation."""
    try:
        # Save the uploaded file temporarily
        temp_filename = f"temp_{uuid.uuid4().hex}{Path(file.filename).suffix}"
        temp_filepath = Path(temp_filename)

        with open(temp_filepath, "wb") as temp_file:
            temp_file.write(await file.read())

        # Extract audio
        audio_path = extract_audio(str(temp_filepath))

        # Transcribe into SRT format text
        srt_text = transcribe_audio_to_srt_format(audio_path)

        # Translate if needed
        if translate:
           srt_text = translate_to_arabic(srt_text)

        # Cleanup
        os.remove(temp_filepath)
        os.remove(audio_path)
        print(srt_text)
        return JSONResponse(content={"transcription": srt_text})

    except subprocess.CalledProcessError as e:
        raise HTTPException(status_code=500, detail=f"FFmpeg error: {e}")
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# **Expose API Using Ngrok**

In [10]:
# Function to run the API in a separate thread
def run_api():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start FastAPI in a background thread
threading.Thread(target=run_api, daemon=True).start()

# Start ngrok tunnel
public_url = ngrok.connect(8000).public_url
print(f"Public URL: {public_url}")

INFO:     Started server process [425]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: https://76c3-34-87-137-39.ngrok-free.app


In [11]:
!curl -X 'POST' \
  'https://9be2-34-85-210-127.ngrok-free.app/transcribe/?translate=true' \
  -H 'accept: application/json' \
  -H 'Content-Type: multipart/form-data' \
  -F 'file=@/content/TestVideo.mp4' > transcription.json

curl: (26) Failed to open/read local data from file/application


In [12]:
# List all available routes
for route in app.routes:
    print(f"Path: {route.path}, Method: {route.methods}")


Path: /openapi.json, Method: {'GET', 'HEAD'}
Path: /docs, Method: {'GET', 'HEAD'}
Path: /docs/oauth2-redirect, Method: {'GET', 'HEAD'}
Path: /redoc, Method: {'GET', 'HEAD'}
Path: /transcribe/, Method: {'POST'}
